<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-libraries-for-Google-Colab-and-Download-Embeddings" data-toc-modified-id="Download-libraries-for-Google-Colab-and-Download-Embeddings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download libraries for Google Colab and Download Embeddings</a></span></li><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Data</a></span></li><li><span><a href="#Load-Embedding-and-Create-Embedding-Layer" data-toc-modified-id="Load-Embedding-and-Create-Embedding-Layer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Embedding and Create Embedding Layer</a></span></li><li><span><a href="#Define-and-Train-Network" data-toc-modified-id="Define-and-Train-Network-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Define and Train Network</a></span></li><li><span><a href="#Pass-Train-and-Test-Data-Through-Network-and-Save-for-Stacking" data-toc-modified-id="Pass-Train-and-Test-Data-Through-Network-and-Save-for-Stacking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pass Train and Test Data Through Network and Save for Stacking</a></span></li></ul></div>

# Download libraries for Google Colab and Download Embeddings

In [0]:
! pip install pandas==0.23.4

    100% |████████████████████████████████| 8.9MB 3.8MB/s 
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [0]:
import pandas as pd
pd.__version__

'0.22.0'

In [0]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2018-12-12 20:04:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-12-12 20:04:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  86.0MB/s    in 9.2s    

2018-12-12 20:04:09 (89.6 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


# Import Data

In [0]:
import numpy as np
import pandas as pd
import csv

In [0]:
pd.__version__

'0.23.4'

In [0]:
df_train = pd.read_csv("train_data.csv",doublequote=True,quotechar='"',sep=",").drop("is_duplicate",axis=1)

In [0]:
df_t_labels= pd.read_csv('train_labels.csv', encoding='utf-8')
df_t_labels.head()

,id,is_duplicate
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [0]:
df_train = df_train.merge(df_t_labels,on=["id"],how="inner")
df_train.head()
df_train['id'] = df_train['id'].apply(str)
df_train['is_duplicate'] = df_train['is_duplicate'].apply(int)

In [0]:
df_train['question1'].fillna('', inplace=True)
df_train['question2'].fillna('', inplace=True)

In [0]:
pd.__version__

'0.23.4'

In [0]:
df_test = pd.read_csv('test_data.csv',doublequote=True)
df_test['test_id'] = df_test['test_id'].apply(str)

In [0]:
df_all = pd.concat((df_train, df_test))
df_all['question1'].fillna('', inplace=True)
df_all['question2'].fillna('', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import itertools

In [0]:
counts_vectorizer = CountVectorizer(max_features=10000-1).fit(
    itertools.chain(df_all['question1'], df_all['question2']))

other_index = len(counts_vectorizer.vocabulary_)

In [0]:
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras 
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
words_tokenizer = re.compile(counts_vectorizer.token_pattern)

In [0]:
# clean an concise vectorised method found on kaggle
def create_padded_seqs(texts, max_len=30):
    seqs = texts.apply(lambda s: 
        [counts_vectorizer.vocabulary_[w] if w in counts_vectorizer.vocabulary_ else other_index
         for w in words_tokenizer.findall(s.lower())])
    return pad_sequences(seqs, maxlen=max_len)

In [0]:
X1_train, X1_val, X2_train, X2_val, y_train, y_val, train_id, train_val_id = \
    train_test_split(create_padded_seqs(df_train['question1']),
                     create_padded_seqs(df_train['question2']),
                     df_train['is_duplicate'].values,
                     df_train.id,
                     stratify=df_train['is_duplicate'].values,
                     test_size=0.3, random_state=2018)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [0]:
train_id_pd = pd.DataFrame(train_id.values)
train_val_id_pd = pd.DataFrame(train_val_id.values)

In [0]:
train_id_pd.to_csv("train_id_merge_dot.csv",index=False)
train_val_id_pd.to_csv("train_val_id_merge_dot.csv", index=False)

# Load Embedding and Create Embedding Layer

In [0]:
import os
import numpy as np

GLOVE_DIR = ""

embeddings_index = {}
line_num = 0

f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'))
ff = list(f)
f.close()

for line in ff:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    line_num += 1
    if line_num == 400000:
        break;

In [0]:
EMBEDDING_DIM = 300

word_index = counts_vectorizer.vocabulary_

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

MAX_SEQUENCE_LENGTH = X1_train.shape[1]

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# Define and Train Network

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
from keras.layers import *
from keras.models import Model

In [0]:
words_embedding_layer = embedding_layer
seq_embedding_layer = LSTM(256, activation='tanh')

input1_tensor = Input(X1_train.shape[1:])
input2_tensor = Input(X2_train.shape[1:])

words_embedding_layer1 = words_embedding_layer(input1_tensor)
words_embedding_layer2 = words_embedding_layer(input2_tensor)

seq_embedding_layer1 = seq_embedding_layer(words_embedding_layer1) 
seq_embedding_layer2 = seq_embedding_layer(words_embedding_layer2)

dot_merge = Dot(axes=1)([seq_embedding_layer1, seq_embedding_layer2])

ouput_layer = Dense(1, activation='sigmoid')(dot_merge)

model = Model([input1_tensor, input2_tensor], ouput_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 300)      3000000     input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
lstm_8 (LSTM)                   (None, 256)          570368      embedding_1[14][0]               
          

In [0]:
# with 300d
#Epoch 00003: val_loss improved from 0.42090 to 0.39988, saving model to weights.best.Siamese.gloverLSTM256
#Epoch 4/5
#226214/226214 [==============================] - 329s 1ms/step - loss: 0.3274 - acc: 0.8537 - val_loss: 0.3956 - val_acc: 0.8191

In [0]:
! mkdir gloverLSTM256.merge.dot

In [0]:
from keras.callbacks import ModelCheckpoint  


directory = "gloverLSTM256.merge.dot/dropout.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"

checkpointer = ModelCheckpoint(filepath=directory, 
                               verbose=1, save_best_only=False,monitor="val_loss")


In [0]:
model.fit([X1_train, X2_train], 
          y_train, 
            batch_size=128, 
            epochs=8,
            callbacks=[checkpointer],
            validation_data=([X1_val, X2_val], y_val))

Train on 226214 samples, validate on 96950 samples
Epoch 1/8
226214/226214 [==============================] - 886s 4ms/step - loss: 0.5888 - acc: 0.6940 - val_loss: 0.5715 - val_acc: 0.7081

Epoch 00001: saving model to gloverLSTM256.merge.dot/dropout.01-0.59-0.57.hdf5
Epoch 2/8
226214/226214 [==============================] - 889s 4ms/step - loss: 0.5383 - acc: 0.7387 - val_loss: 0.5311 - val_acc: 0.7434

Epoch 00002: saving model to gloverLSTM256.merge.dot/dropout.02-0.54-0.53.hdf5
Epoch 3/8
226214/226214 [==============================] - 893s 4ms/step - loss: 0.5137 - acc: 0.7531 - val_loss: 0.5208 - val_acc: 0.7495

Epoch 00003: saving model to gloverLSTM256.merge.dot/dropout.03-0.51-0.52.hdf5
Epoch 4/8
226214/226214 [==============================] - 682s 3ms/step - loss: 0.4921 - acc: 0.7667 - val_loss: 0.5151 - val_acc: 0.7538

Epoch 00004: saving model to gloverLSTM256.merge.dot/dropout.04-0.49-0.52.hdf5
Epoch 5/8
226214/226214 [==============================] - 601s 3ms/step 

In [0]:
#gloverd100
#dropout
#multi-merge

#Train on 226214 samples, validate on 96950 samples
#Epoch 1/5
#226214/226214 [==============================] - 944s 4ms/step - loss: 0.4946 - acc: 0.7509 - val_loss: 0.4351 - val_acc: 0.7880
#
#Epoch 00001: val_loss improved from inf to 0.43513, saving model to weights.best.Siamese.gloverLSTM256.multiple.merge.dropout
#Epoch 2/5
#226214/226214 [==============================] - 818s 4ms/step - loss: 0.4106 - acc: 0.8037 - val_loss: 0.4029 - val_acc: 0.8086
#
#Epoch 00002: val_loss improved from 0.43513 to 0.40287, saving model to weights.best.Siamese.gloverLSTM256.multiple.merge.dropout
#Epoch 3/5
#226214/226214 [==============================] - 886s 4ms/step - loss: 0.3725 - acc: 0.8258 - val_loss: 0.3892 - val_acc: 0.8173
#
#Epoch 00003: val_loss improved from 0.40287 to 0.38922, saving model to weights.best.Siamese.gloverLSTM256.multiple.merge.dropout
#Epoch 4/5
#226214/226214 [==============================] - 813s 4ms/step - loss: 0.3392 - acc: 0.8447 - val_loss: 0.3821 - val_acc: 0.8215
#
#Epoch 00004: val_loss improved from 0.38922 to 0.38209, saving model to weights.best.Siamese.gloverLSTM256.multiple.merge.dropout
#Epoch 5/5
#226214/226214 [==============================] - 941s 4ms/step - loss: 0.3074 - acc: 0.8613 - val_loss: 0.3796 - val_acc: 0.8283
#
#Epoch 00005: val_loss improved from 0.38209 to 0.37959, saving model to weights.best.Siamese.gloverLSTM256.multiple.merge.dropout
#<keras.callbacks.History at 0x7fa0450c7550>

#Train on 226214 samples, validate on 96950 samples
#Epoch 1/3
#226214/226214 [==============================] - 617s 3ms/step - loss: 0.2778 - acc: 0.8767 - val_loss: 0.3832 - val_acc: 0.8305
#
#Epoch 00001: val_loss did not improve from 0.37959
#Epoch 2/3
#226214/226214 [==============================] - 475s 2ms/step - loss: 0.2490 - acc: 0.8909 - val_loss: 0.3884 - val_acc: 0.8309
#
#Epoch 00002: val_loss did not improve from 0.37959
#Epoch 3/3
#226214/226214 [==============================] - 337s 1ms/step - loss: 0.2223 - acc: 0.9041 - val_loss: 0.4159 - val_acc: 0.8352
#
#Epoch 00003: val_loss did not improve from 0.37959
#<keras.callbacks.History at 0x7fa044fdaac8>

In [0]:
# Load Best Epoch
model.load_weights("gloverLSTM256.merge.dot/dropout.04-0.49-0.52.hdf5")

# Pass Train and Test Data Through Network and Save for Stacking

In [0]:
embeddingsfeatures_model_dot = Model([input1_tensor, input2_tensor], dot_merge)
features_model_dot.compile(loss='mse', optimizer='adam')

In [0]:
train_preds = [create_padded_seqs(df_train['question1']),create_padded_seqs(df_train['question2'])]
test_preds = [create_padded_seqs(df_test['question1']),create_padded_seqs(df_test['question2'])]

In [0]:
F_train_dot = features_model_dot.predict(train_preds, batch_size=128)
F_test_dot = features_model_dot.predict(test_preds, batch_size=128)

In [0]:
F_train_1 = model.predict(train_preds, batch_size=128)
F_test_1 = model.predict(test_preds, batch_size=128)

In [0]:
test_1 = pd.concat([df_test.test_id,pd.DataFrame(F_test_1)],axis=1)
test_dot = pd.concat([df_test.test_id,pd.DataFrame(F_test_dot)],axis=1)

In [0]:
print(test_1.shape)
print(df_test.shape)
test_dot.head(1)

(81126, 2)
(81126, 3)


,test_id,0
0,15,0.061864


In [0]:
train_1 = pd.concat([df_train.id,pd.DataFrame(F_train_1)],axis=1)
train_dot = pd.concat([df_train.id,pd.DataFrame(F_train_dot)],axis=1)

In [0]:
print(train_1.shape)
print(df_train.shape)
train_dot.head(1)

(323164, 2)
(323164, 4)


,id,0
0,0,3.164393


In [0]:
train_dot.to_csv("train_dot_layer_merge_dot.csv",index=False)
train_1.to_csv("train_output_merge_dot.csv",index=False)

In [0]:
pd.read_csv("train_dot_layer_merge_dot.csv").head()

,id,0
0,0,3.164393
1,1,5.175281
2,2,4.425738
3,3,5.461930
4,4,5.053749


In [0]:
test_dot.to_csv("test_dot_layer_merge_dot.csv",index=False)
test_1.to_csv("test_output_merge_dot.csv",index=False)

In [0]:
pd.read_csv("train_1.csv").head()

,id,0
0,0,0.654116
1,1,0.020497
2,2,0.078235
3,3,0.001608
4,4,0.007873
